In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
else:
    print(gpu_info)

Thu Jun  9 17:52:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 511.79       Driver Version: 511.79       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   56C    P8     8W /  N/A |   7885MiB /  8192MiB |      9%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from PIL import Image
from tqdm import tqdm
import numpy as np
from tensorflow.keras.optimizers import Adam
from tqdm import tqdm
from keras.initializers import RandomNormal
from keras.models import Model, Input, load_model
from keras.layers import Conv2D, Conv2DTranspose, LeakyReLU, Activation, Concatenate ,Dropout, BatchNormalization
from keras.preprocessing import image
from keras.utils.vis_utils import plot_model
import os
from os import listdir
from os.path import join, isfile
import tensorflow as tf
from csv import writer
from matplotlib import pyplot

In [3]:
print(os.getcwd())

C:\Users\Ahmed\Downloads\edges2shoes\edges2shoes


# Loading Dataset

In [4]:
def load_image(path, size = (256,256)):
    imgs_list = list()
    for filename in tqdm(listdir(path)):
        img = image.load_img(path + filename, target_size=size)      
        img = image.img_to_array(img)
        imgs_list.append(img)
    return [np.asarray(imgs_list)]

In [5]:
[src_images] = load_image('bags_train_s/')
[tar_images] = load_image('bags_train_i/')

100%|███████████████████████████████████████████████████████████████████████████| 12000/12000 [00:19<00:00, 615.55it/s]


In [6]:
src_images.shape

(12000, 256, 256, 3)

In [7]:
tar_images.shape

(12000, 256, 256, 3)

# Discriminator

In [8]:
def discriminator(image_shape):

    init = RandomNormal(stddev=0.02)

    in_src_image = Input(shape=image_shape)

    in_target_image = Input(shape=image_shape)

    merged = Concatenate()([in_src_image, in_target_image])

    d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
    d = LeakyReLU(alpha=0.2)(d)

    d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)

    d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)

    d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)

    d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)

    d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
    patch_out = Activation('sigmoid')(d)

    model = Model([in_src_image, in_target_image], patch_out)

    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
    return model

# Encoder Block

In [9]:
def encoder_block(layer_in, n_filters, batchnorm=True):
    init = RandomNormal(stddev=0.02)
    g = Conv2D(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
    if batchnorm:
        g = BatchNormalization()(g, training=True)
    g = LeakyReLU(alpha=0.2)(g)
    return g

# Decoder Block

In [10]:
def decoder_block(layer_in, skip_in, n_filters, dropout=True):
    init = RandomNormal(stddev=0.02)
    g = Conv2DTranspose(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
    g = BatchNormalization()(g, training=True)

    if dropout:
        g = Dropout(0.5)(g, training=True)
    #print(g.shape)
    #print(skip_in.shape)
    g = Concatenate()([g, skip_in])
    g = Activation('relu')(g)
    return g

# Generator

In [11]:
def generator(image_shape=(256,256,3)):
    init = RandomNormal(stddev=0.02)
    in_image = Input(shape=image_shape)
    # encoder model
    e1 = encoder_block(in_image, 64, batchnorm=False)
    e2 = encoder_block(e1, 128)
    e3 = encoder_block(e2, 256)
    e4 = encoder_block(e3, 512)
    e5 = encoder_block(e4, 512)
    e6 = encoder_block(e5, 512)
    e7 = encoder_block(e6, 512)
    # bottleneck, no batch norm and relu
    b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(e7)
    b = Activation('relu')(b)
    # decoder model
    d1 = decoder_block(b, e7, 512)
    d2 = decoder_block(d1, e6, 512)
    d3 = decoder_block(d2, e5, 512)
    d4 = decoder_block(d3, e4, 512, dropout=False)
    d5 = decoder_block(d4, e3, 256, dropout=False)
    d6 = decoder_block(d5, e2, 128, dropout=False)
    d7 = decoder_block(d6, e1, 64, dropout=False)
    # output
    g = Conv2DTranspose(3, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d7)
    out_image = Activation('tanh')(g)
    # define model
    model = Model(in_image, out_image)
    return model

# Gan model

In [12]:
def gan(g_model, d_model, image_shape):
    d_model.trainable = False
    in_src = Input(shape=image_shape)
    gen_out = g_model(in_src)
    dis_out = d_model([in_src, gen_out])
    model = Model(in_src, [dis_out, gen_out])
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1,100])
    return model

In [13]:
def generate_real_samples(dataset, n_samples, patch_shape):
    # unpack dataset
    trainA, trainB = dataset
    ix = np.random.randint(0, trainA.shape[0], n_samples)
    X1, X2 = trainA[ix], trainB[ix]
    y = np.ones((n_samples, patch_shape, patch_shape, 1))
    return [X1, X2], y

In [14]:
def generate_fake_samples(g_model, samples, patch_shape):
    X = g_model.predict(samples)
    y = np.zeros((len(X), patch_shape, patch_shape, 1))
    return X, y

In [15]:
# Print directory
print(os.getcwd())

C:\Users\Ahmed\Downloads\edges2shoes\edges2shoes


# Summarizing Performance

In [16]:
def summarize_performance(step, g_model, dataset, n_samples=3):
    [X_realA, X_realB], _ = generate_real_samples(dataset, n_samples, 1)
    X_fakeB, _ = generate_fake_samples(g_model, X_realA, 1)
    X_realA = (X_realA + 1) / 2.0
    X_realB = (X_realB + 1) / 2.0
    X_fakeB = (X_fakeB + 1) / 2.0

    for i in range(n_samples):
        pyplot.subplot(3, n_samples, 1 + i)
        pyplot.axis('off')
        pyplot.imshow(X_realA[i])

    for i in range(n_samples):
        pyplot.subplot(3, n_samples, 1 + n_samples + i)
        pyplot.axis('off')
        pyplot.imshow(X_fakeB[i])
    # plot real target image
    for i in range(n_samples):
        pyplot.subplot(3, n_samples, 1 + n_samples*2 + i)
        pyplot.axis('off')
        pyplot.imshow(X_realB[i])
    # save plot to file
    filename1 = 'tests/plot_ck_%07d.png' % (step+1)
    pyplot.savefig(filename1)
    pyplot.close()
    print('>Saved: %s' % (filename1))

# Train GAN model

In [17]:
def train(d_model, g_model, gan_model, dataset, n_epochs=35, n_batch=8):
    
    generator_optimizer = Adam(learning_rate = 0.0002, beta_1=0.5)
    discriminator_optimizer = Adam(learning_rate = 0.0002, beta_1=0.5)
    checkpoint_dir = 'Bags_Checkpoints'
    checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
    checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer, 
                                     discriminator_optimizer=discriminator_optimizer, 
                                     generator=g_model, discriminator=d_model, 
                                     gan_model = gan_model)
    ckpt_manager = tf.train.CheckpointManager(checkpoint, checkpoint_dir, max_to_keep=3)
    if ckpt_manager.latest_checkpoint:
        checkpoint.restore(ckpt_manager.latest_checkpoint)  #ckpt_manager.checkpoints[3]
        print('Latest checkpoint restored!')
    else:
        print('No Previous Checkpoints')
    
    n_patch = d_model.output_shape[1]
    trainA, trainB = dataset
    bat_per_epo = int(len(trainA) / n_batch)
    print("Batch Per Epoch: {}".format(bat_per_epo))
    print("Length of TrainA: {}".format(len(trainA)))
    n_steps = bat_per_epo * n_epochs
    for i in tqdm(range(n_steps)):
        [X_realA, X_realB], y_real = generate_real_samples(dataset, n_batch, n_patch)
        X_fakeB, y_fake = generate_fake_samples(g_model, X_realA, n_patch)
        d_loss1 = d_model.train_on_batch([X_realA, X_realB], y_real)
        d_loss2 = d_model.train_on_batch([X_realA, X_fakeB], y_fake)
        g_loss, _, _ = gan_model.train_on_batch(X_realA, [y_real, X_realB])
        if i % 300 == 0:
            print('>%d, d1[%.5f] d2[%.5f] g[%.5f]' % (i+1, d_loss1, d_loss2, g_loss))
            with open('logs.csv', 'a', newline='') as f_object:  
                writer_object = writer(f_object)
                writer_object.writerow([i+1, d_loss1, d_loss2, g_loss])  
                f_object.close()
        if (i+1) % (bat_per_epo * 2) == 0:
            summarize_performance(i, g_model, dataset)
            ckpt_save_path = ckpt_manager.save()
            print ('Saving checkpoint for epoch {} at {}'.format(i+1,ckpt_save_path))
            g_model.save('Bags_Checkpoints/model_Bags_%07d.h5' % (i+1))
            print('>Saved g_model: %s ' % ('model_Bags_%07d.h5' % (i+1)))
            print('Epoch %d' % int(i/3000))

            

# Scaling Real and Sketch images

In [21]:
src_images = (src_images - 127.5) / 127.5


In [22]:
tar_images = (tar_images - 127.5) / 127.5

# Main

In [ ]:
dataset = [src_images, tar_images]
print('Loaded', dataset[0].shape, dataset[1].shape)
image_shape = dataset[0].shape[1:]
# define the models
d_model = discriminator(image_shape)
g_model = generator(image_shape)
# define the gan model
gan_model = gan(g_model, d_model, image_shape)
# train model
train(d_model, g_model, gan_model, dataset)

Loaded (12000, 256, 256, 3) (12000, 256, 256, 3)


C:\Users\Ahmed\anaconda3\envs\gpu_env\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Latest checkpoint restored!
Batch Per Epoch: 1500
Length of TrainA: 12000


  0%|                                                                            | 1/52500 [00:18<263:26:35, 18.07s/it]

>1, d1[0.00000] d2[0.00000] g[20.05909]


...

 99%|█████████████████████████████████████████████████████████████████████████▏| 51901/52500 [7:13:09<04:55,  2.03it/s]

>51901, d1[0.00000] d2[0.00000] g[6.28777]


 99%|█████████████████████████████████████████████████████████████████████████▌| 52201/52500 [7:15:38<02:26,  2.04it/s]

>52201, d1[0.00000] d2[0.00000] g[5.77962]


100%|█████████████████████████████████████████████████████████████████████████▊| 52370/52500 [7:17:07<01:08,  1.89it/s]

In [2]:
'''
num = 6000
mpath = 'Checkpoints/model_ck_000'
gt = False
for i in range(7):
    if num >= 9999 and not gt:
        mpath = mpath[:-1]
        gt = True
    mm = load_model(mpath+ str(num) + '.h5')
    summarize_performance(num-1, mm, dataset)
    num += 3000
    '''
    

"\nnum = 6000\nmpath = 'Checkpoints/model_ck_000'\ngt = False\nfor i in range(7):\n    if num >= 9999 and not gt:\n        mpath = mpath[:-1]\n        gt = True\n    mm = load_model(mpath+ str(num) + '.h5')\n    summarize_performance(num-1, mm, dataset)\n    num += 3000\n    "

In [ ]:
#model.summary()

In [ ]:
#plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
#!pip install pydot 
